# <center> 1. Знакомимся с данными</center>

✍ Вы уже умеете присоединять строки друг к другу путём добавления столбцов «сбоку» — с помощью различных видов JOIN.

? А что если нам необходимо присоединить несколько результатов «снизу», так, чтобы получить общий результат в выводе?

Ответу на этот вопрос и посвящён текущий модуль.

Обратите внимание! Вы можете посмотреть подробный разбор структуры предлагаемых запросов во вкладке Детализация.
Но прежде, как всегда, разберёмся с датасетом ↓

В данном блоке мы будем работать с данными о компании, организующей перевозки грузов.

Интересующие нас данные хранятся в таблицах city, customer, driver, shipment, truck. Давайте внимательно их изучим.

Ниже представлена ER-диаграмма (от англ. entity-relation, дословно — «сущность-связь»), которая отображает существующие связи между отдельными таблицами.

![](data/dst3-u2-md4_1_1.jpg)

Таблица city — это справочник городов. Структура справочника представлена ниже.

|Название поля	|Тип данных	|Описание|
|---|---|---|
|city_id	|integer	|уникальный идентификатор города, первичный ключ|
|city_name	|text	|название города|
|state	|text	|штат, к которому относится город|
|population	|integer	|население города|
|area	|numeric	|площадь города|

Таблица customer — это справочник клиентов. У компании, с данными которой мы работаем, только корпоративные клиенты, поэтому в таблице нет привычных данных о возрасте и поле. Справочник содержит следующие поля:

|Название поля	|Тип данных	|Описание|
|---|---|---|
|driver_id	|integer	|уникальный идентификатор водителя, первичный ключ|
|first_name	|text	|имя водителя|
|last_name	|text	|фамилия водителя|
|address	|text	|адрес водителя|
|zip_code	|integer	|почтовый индекс водителя|
|phone	|text	|телефон водителя|
|city_id	|integer	|идентификатор города водителя, внешний ключ к таблице city|

В таблице truck хранится информация о грузовиках, на которых осуществляются перевозки. Данные о них представлены в следующем виде:

|Название поля	|Тип данных	|Описание|
|---|---|---|
|truck_id	|integer	|Уникальный идентификатор грузовика, первичный ключ|
|make	|text	|Производитель грузовика|
|model_year	|integer	|Дата выпуска грузовика|

Последняя таблица в датасете, shipment, — таблица с данными непосредственно о доставках. Она описывает взаимодействие всех перечисленных сущностей, а потому содержит наибольшее количество ссылок на другие таблицы.

|Название поля	|Тип данных	|Описание|
|---|---|---|
|ship_id	|integer	|уникальный идентификатор доставки, первичный ключ|
|cust_id	|integer	|идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer|
|weight	|numeric	|вес посылки|
|truck_id	|integer	|идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck|
|driver_id	|integer	|идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver|
|city_id	|integer	|идентификатор города в который совершена доставка, внешний ключ к таблице city|
|ship_date	|date	|дата доставки|

✍ Исследуйте датасет самостоятельно и ответьте на вопросы ниже ↓

→ Укажите название города с максимальным весом единичной доставки.

```sql
SELECT city_name, MAX(weight)
FROM sql.city c
JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY city_name
ORDER BY MAX(weight) DESC
LIMIT 1
```

→ Сколько различных производителей грузовиков перечислено в таблице truck?

```sql
SELECT COUNT(DISTINCT make) FROM sql.truck
```

→ Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

```sql
SELECT dr.first_name, sh.cust_id, COUNT(sh.ship_id)
FROM sql.shipment sh
JOIN sql.driver dr ON  dr.driver_id=sh.driver_id
GROUP BY dr.first_name, sh.cust_id
ORDER BY COUNT(sh.ship_id) DESC, dr.first_name
```

→ Укажите даты первой и последней по времени доставок в таблице shipment.

```sql
SELECT MAX(ship_date), MIN(ship_date)
FROM sql.shipment
```

→ Укажите имя клиента, получившего наибольшее количество доставок за 2017 год.

```sql
SELECT cust.cust_name, COUNT(sh.ship_id)
FROM sql.customer AS cust
JOIN sql.shipment AS sh ON cust.cust_id=sh.cust_id
GROUP BY cust.cust_name
ORDER BY COUNT(sh.ship_id) DESC
LIMIT 1
```

# <center>2. UNION</center>

## <center>Принцип и условия работы Union</center>

✍ Вернёмся к центральному вопросу модуля: как соединить несколько результатов, чтобы получить в выводе один общий?

Чтобы разобраться в этом вопросе, смоделируем ситуацию.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем простой запрос:

```sql
SELECT book_name object_name, 'книга' object_descritption /*выбираем столбец с названием book_name, задаём алиас для столбца object_name, задаём во второй колонке объект ‘книга’ с алиасом для столбца object_descritption*/
FROM public.books /*из схемы public и таблицы books*/
UNION ALL /*оператор присоединения*/
SELECT movie_title, 'фильм' /*выбираем столбец movie_title, сами задаём во второй колонке объект ‘фильм’*/
FROM sql.kinopoisk /*из схемы sql и таблицы kinopoisk*/
```

Визуально произведённое нами действие можно представить следующим образом:  
![](data/dst3-u2-md4_2_1.png)

Общий принцип мы поняли, разберёмся в деталях:

В запросе мы использовали оператор UNION ALL — он присоединяет любой результат запроса к другому «снизу» при условии, что у них одинаковая структура, а именно:  
* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

## <center>Виды UNION</center>

Оператор присоединения существует в двух вариантах:  
* UNION выводит только уникальные записи;
* UNION ALL присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

**Важно! UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать UNION ALL.
Проверить, как различаются операторы, вы сможете при выполнении заданий 2.1 и 3.1.**

Синтаксис

Запрос строится таким образом:
```sql
SELECT         n columns
FROM 
         table_1
UNION ALL
SELECT 
         n columns
FROM 
         table_2
...
UNION ALL
SELECT 
         n columns
FROM 
         table_n
```
Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.  
Важно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

ажно! Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.
Пришла пора испытать функцию UNION(ALL) на практике.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.  
```sql
SELECT
         c.city_id object_name,  'id города' object_type /*выбираем колонку city_id и задаём ей алиас object_name, сами задаём объект 'id города' и название столбца object_type*/
FROM 
         sql.city c /*из схемы sql и таблицы city, задаём алиас таблице — с*/
UNION ALL /*оператор присоединения*/
SELECT
         d.driver_id other_name,  'id водителя' other_type /*выбираем колонку driver_id и задаём ей алиас other_name, сами задаём объект 'id водителя' и название столбца other_type*/
FROM 
         sql.driver d  /*из схемы sql и таблицы driver, задаём алиас таблице — d*/
UNION ALL /*оператор присоединения*/
SELECT
         s.ship_id,  'id доставки' /*выбираем колонку ship_id, сами задаём объект 'id доставки'*/
FROM 
         sql.shipment s /*из схемы sql и таблицы shipment, задаём алиас таблице — s*/
UNION ALL /*оператор присоединения*/
SELECT
         c.cust_id,  'id клиента' /*выбираем колонку cust_id, сами задаём объект 'id клиента'*/
FROM 
         sql.customer c /*из схемы sql и таблицы customer, задаём алиас таблице — c*/
UNION ALL /*оператор присоединения*/
SELECT
         t.truck_id,  'id грузовика' /*выбираем колонку truck_id, сами задаём объект 'id грузовика'*/
FROM 
         sql.truck t /*из схемы sql и таблицы truck, задаём алиас таблице — t*/
ORDER BY 1 /*сортировка по первому столбцу*/
```

Обратите внимание! Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.  
Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.  
В случаях, когда необходимо применить команду ORDER BY или LIMIT не к итоговому результату, а к каждой части запроса, можно обернуть подзапросы в скобки.  
Чтобы посмотреть, как это работает, вернёмся к нашему примеру с общим справочником по фильмам и книгам.  
Мы уже знаем, что можно легко и непринуждённо применить операторы ORDER BY и LIMIT ко всему результату запроса.

```sql
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
```

Всё бы хорошо, только в таком случае отсортирован будет весь общий справочник, а в выводе останется одна строка с названием объекта, идущим первым по алфавиту.  
А если мы не хотим общую сортировку? Может, нам нужны строки с названием как фильма, так и книги, идущих первыми по алфавиту.  
Нет ничего проще — отсортируем каждую часть запроса по отдельности и объединим результаты!  
Просто добавим ORDER BY и LIMIT ещё и в первую часть запроса:

```sql
SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1
UNION ALL
SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1
```

Вместо результата получим сообщение о синтаксической ошибке: "...syntax error at or near "UNION"..." Очевидно, этот фокус не удался.  
Не стоит огорчаться, ведь проблему можно решить одним (ну, почти) движением руки — просто добавив скобки вокруг каждой из частей запроса.

```sql
(SELECT book_name object_name, 'книга' object_descritption 
FROM public.books
ORDER BY 1
LIMIT 1)
UNION ALL
(SELECT movie_title, 'фильм' 
FROM sql.kinopoisk
ORDER BY 1
LIMIT 1)
```  
Отлично! Мы получили именно то, что хотели.

Напишите запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). Отсортируйте список по названию объекта, а затем — по типу.

```sql
SELECT c.city_name "название", 'city' "тип объекта"
FROM sql.city c
UNION
SELECT c.state, 'state'
FROM sql.city c
UNION
SELECT d.first_name, 'driver'
FROM sql.driver d
UNION
SELECT t.make, 'truck'
FROM sql.truck t
ORDER BY 1,2
```

Напишите запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

```sql
SELECT c.city_name object_name
FROM sql.city c
UNION ALL
SELECT c.state
FROM sql.city c
ORDER BY 1
```

Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

```sql
SELECT c.city_name object_name
FROM sql.city c
UNION
SELECT c.state
FROM sql.city c
ORDER BY 1
```

# <center> 3. UNION и ограничение типов данных</center>

## <center>Почему так важен тип данных?</center>

Как мы уже знаем, UNION может быть использован только в случае полного соответствия столбцов и их типов в объединяемых запросах.  
Допустим, мы хотим вывести список всех id городов и их названий в одном столбце.

Давайте напишем запрос, который позволит получить нужный нам результат.  
```sql
SELECT 
         c.city_id /*выбираем столбец city_id*/
FROM
         sql.city c /*из схемы sql  и таблицы city, задаём таблице алиас с*/ 

UNION ALL /*оператор присоединения*/

SELECT 
         cc.city_name /*выбираем столбец city_name*/
FROM
         sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас сс*/
```

Вместо результата вы получите сообщение об ошибке: "ERROR: UNION types integer and text cannot be matched". Дело в том, что мы попытались объединить числовой и строковый типы в одной колонке, а это невозможно.  
Если мы всё же хотим выполнить поставленную задачу, придётся привести оба столбца к одному типу данных. Не каждый текст может быть приведён к числу, зато каждое число может быть представлено в текстовом формате.  
Забегая вперёд, скажем пару слов о типизации столбцов. Для типизации в Postgres составляется запрос по модели column_name::column_type.  
Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.  
Важно! Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

Немного подправим запрос, чтобы получить желаемый результат.

```sql
SELECT 
         c.city_id::text /*выбираем столбец city_id, переводим city_id из числового в текстовый формат*/
FROM
         sql.city c /*из схемы sql  и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT 
         cc.city_name /*выбираем столбец city_name*/
FROM
         sql.city cc /*из схемы sql и таблицы city, задаём таблице алиас сс*/
```

Напишите запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

```sql
SELECT
    d.zip_code::text contact,
    d.first_name first_name,
    'zip' contact_type
FROM
    sql.driver d
UNION
SELECT
    dd.phone contact,
    dd.first_name first_name,
    'phone' contact_type
FROM
    sql.driver dd
ORDER BY 1,2

```

# <center>4. UNION ALL и промежуточные итоги</center>

## <center>Возможности UNION</center>

✍ Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

Попробуем вывести обобщённые данные о населении по всем городам, с детализацией до конкретного города.  
```sql
SELECT
         c.city_name,
         c.population /*выбираем столбцы city_name, population*/
FROM
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/

UNION ALL /*оператор присоединения*/

SELECT
         'total',
         SUM(c.population) /*сами задаём объект ‘total’, суммируем все значения столбца population*/
FROM
         sql.city c /*из схемы sql и таблицы city, задаём таблице алиас с*/
ORDER BY 2 DESC /*сортируем по второму столбцу в убывающем порядке (чтобы итоговая сумма была в начале)*/
```  
Визуально это действие можно представить так:  
![](data/dst3-u2-md4_4_1.png)  
Выполнив запрос в Metabase, вы заметите, что сначала выведен итоговый результат с суммой населения всех городов в таблице, а затем по каждому городу в отдельности.

Напишите запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. Упорядочите по убыванию столбца date_period.  
```sql
SELECT s.ship_date::text date_period, COUNT(*) cnt_shipment
FROM sql.shipment s 
GROUP BY 1
UNION ALL
SELECT 'total_shipments', COUNT(*)
FROM sql.shipment s
ORDER BY 1 desc
```

# <center>5. UNION и дополнительные условия</center>

✍ UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».  
Например, с помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона.  
```sql
SELECT
         d.first_name,
         d.last_name,
         'телефон заполнен' phone_info /*выбираем столбцы first_name, last_name, сами выводим объект ‘телефон заполнен’*/
FROM
         sql.driver d /*из схемы sql и таблицы driver, задаём алиас d*/
WHERE d.phone IS NOT NULL /*условие, что телефон заполнен*/

UNION /*оператор присоединения (уникальные значения)*/

SELECT
         d.first_name,
         d.last_name,
         'телефон не заполнен' phone_info /*выбираем столбцы first_name, last_name, сами выводим объект ‘телефон не заполнен’*/
FROM
         sql.driver d /*из схемы sql и таблицы driver, задаём алиас d*/
WHERE d.phone IS NULL /*условие, что телефон не заполнен*/
```

Напишите запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:

если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
если нет — выводим 'доставка не осуществлялась'.
Столбцы к выводу: city_name, state, shipping_status. Отсортируйте в алфавитном порядке по городу, а затем — по штату.

```sql
SELECT 
     c.city_name AS city_name,
     c.state AS state,
    'доставка осуществлялась' AS shipping_status
FROM 
    sql.city c
    LEFT JOIN sql.shipment s ON c.city_id=s.city_id
WHERE s.city_id IS NOT NULL
UNION
SELECT 
     c.city_name AS city_name,
     c.state AS state,
    'доставка не осуществлялась' AS shipping_status
FROM 
    sql.city c
    LEFT JOIN sql.shipment s ON c.city_id=s.city_id
WHERE s.city_id IS NULL
ORDER BY 1, 2
```

Напишите запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:

если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
иначе — вывести количество доставок, увеличенное на пять.
Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.  
```sql
SELECT
    c.city_name AS city_name,
    COUNT(s.ship_id) shippings_fake
FROM
    sql.city c
    JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY
    c.city_name
HAVING
     COUNT(s.ship_id) > 10
UNION
SELECT
    c.city_name AS city_name,
    COUNT(s.ship_id)+5 shippings_fake
FROM
    sql.city c
    JOIN sql.shipment s ON c.city_id=s.city_id
GROUP BY
    c.city_name
HAVING
     COUNT(s.ship_id) <= 10
ORDER BY
    shippings_fake desc,
    city_name asc

```

# <center> 6. UNION и ручная генерация</center>

✍ UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.  
Составим запрос, который позволит вывести первые три буквы алфавита и их порядковые номера.  
```sql
SELECT 
         'a' letter,'1' ordinal_position /*сами задаём значение первого столбца ‘a’ и алиас для него letter, значение второго столбца ‘1’ и алиас для него ordinal_position*/
         
UNION /*оператор присоединения*/
SELECT 
         'b','2' /*сами задаём значение первого столбца ‘b’, значение второго столбца ‘2’ */
         
UNION /*оператор присоединения*/

SELECT
         'c','3' /*сами задаём значение первого столбца ‘с’, значение второго столбца ‘3’*/ 
```  
Существуют сложные алгоритмы сравнения текстовых значений, но главный смысл сводится к одному: сравнение производится на основе таблицы unicode и позиции элемента в ней с учётом определённых условий.

✍ Упражнения ниже помогут увидеть разницу в сравнении числовых и текстовых значений ↓

Напишите запрос, который выберет наибольшее из числовых значений:

1000000;
541;
-500;
100.
Столбец с результатом назовите result.  
```sql
SELECT  1000000 result
UNION ALL
SELECT  541
UNION ALL
SELECT  -500
UNION ALL
SELECT  100
ORDER BY 1 DESC 
LIMIT 1
```

Помним, что сортировка для числовых и строковых типов данных отличается. Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

1000000;
541;
-500;
100.
Столбец с ответом назовите mycol  
```sql
SELECT  '1000000' as mycol
UNION ALL
SELECT  '541' 
UNION ALL
SELECT  '-500'
UNION ALL
SELECT  '100'
ORDER BY 1 DESC 
LIMIT 1
```

Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

+ ;
- ;
= ;
/ .
Столбец с ответом назовите result.  
```sql
SELECT '+' as result
UNION ALL
SELECT '-'
UNION ALL
SELECT '='
UNION ALL
SELECT '/'
ORDER BY 1 DESC
LIMIT 1
```

# <center>7. EXCEPT</center>

## <center>Исключаем повторяющиеся данные</center>

Предположим, нам нужно узнать, в какие города осуществлялась доставка, за исключением тех, в которых проживают водители.  
```sql
SELECT
         c.city_name /*выбираем столбец city_name*/
FROM
         sql.shipment s /*из схемы sql и таблицы shipment, задаём таблице алиас s*/
JOIN sql.city c ON s.city_id = c.city_id /*внутреннее присоединение из схемы sql таблицы city, задав ей алиас c, по ключам city_id*/
EXCEPT /*оператор присоединения*/
SELECT
         cc.city_name  /*выбираем столбец city_name*/
FROM
         sql.driver d /*из схемы sql и таблицы driver, задаём таблице алиас d*/
JOIN sql.city cc ON d.city_id=cc.city_id /*внутреннее присоединение из схемы sql таблицы city, задав ей алияс cc, по ключам city_id*/
ORDER BY 1 /*сортировка по первому столбцу*/
```

Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.  
Как вы, должно быть, заметили, для решения этой задачи мы использовали оператор EXCEPT. Принципы его работы мы сейчас разберём ↓  
Чтобы лучше понять данный тип присоединения, предлагаем ознакомиться с диаграммой Венна — математическим инструментом, представляющим возможные логические связи между соединёнными наборами данных.  
![](data/dst3-u2-md4_7_1.png)  
Синтаксические правила для оператора EXCEPT такие же, как и для UNION:  
* одинаковый тип данных;
* одинаковое количество столбцов;
* одинаковый порядок столбцов согласно типу данных.

Синтаксис выглядит следующим образом:
```sql
SELECT 
         n columns
FROM 
         table_1
EXCEPT
SELECT 
         n columns
FROM 
         table_2
```
Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.  
Предположим, у нас есть информация о продажах канцелярского магазина за май и июнь.  
Какие-то позиции продавались и в том, и в другом месяце, а какие-то — только в одном. Использовав EXCEPT, мы можем оставить только те товары, которые есть в первом запросе (например, за май), но отсутствуют во втором запросе (например, за июнь).  
Графически действие оператора можно представить следующим образом:  
![](data/dst3-u2-md4_7_2.png)  
Таким образом, при присоединении с помощью EXCEPT мы вывели только те товары, которые были проданы в мае, но не в июне. Чтобы найти продажи по тем позициям, что были реализованы в июне, а в мае — нет, необходимо поменять запросы местами.

Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. Отсортируйте по возрастанию, столбец к выводу — zip. В поле ниже введите запрос, с помощью которого вы решили эту задачу.  
```sql
SELECT zip_code AS zip
FROM sql.driver
EXCEPT 
SELECT zip 
FROM sql.customer
ORDER BY 1
```

# <center>8. INTERSECT</center>